# Аналитический отчёт по задаче прогнозирования целевого действия

**Цель проекта:**  
Построить модель машинного обучения, прогнозирующую вероятность совершения целевого действия пользователем в рамках сессии на сайте.

**Данные:**  
- `ga_sessions.pkl` — данные о визитах пользователей  
- `ga_hits.csv` — данные о событиях внутри визитов (агрегированы до уровня сессии)
  

Комментарий: 
Этот код выполняет загрузку двух наборов данных для последующего анализа.

Загрузка данных сессий (sessions) из pickle-формата - бинарного формата хранения объектов Python

Загрузка агрегированных данных о взаимодействиях (hits_agg) из Parquet-формата - колоночного формата, оптимизированного для аналитических запросов

Вывод размерности каждого датафрейма, что помогает оценить объем данных перед их обработкой

In [ ]:
import pandas as pd

print("Читаю данные...")
# Загрузка данных сессий из pickle файла
sessions = pd.read_pickle("../ga_sessions.pkl")
# Загрузка агрегированных данных хитов из parquet файла
hits_agg = pd.read_parquet("../hits_agg.parquet")

# Вывод размерности (количество строк и столбцов) каждого датафрейма
print("Размер sessions:", sessions.shape)  # Формат: (количество_строк, количество_столбцов)
print("Размер hits_agg:", hits_agg.shape)  # Формат: (количество_строк, количество_столбцов)

Комментарий: Этот этап включает объединение данных и предобработку целевой переменной.

Если доля target=1 очень мала (<5%), это задача с несбалансированными классами

Если доля около 50%, данные хорошо сбалансированы

Значение помогает выбрать стратегию построения модели и её оценки

In [ ]:
print("Объединяю данные...")
# Объединение таблиц sessions и hits_agg по ключевому полю session_id
# how="left" означает левое соединение: все строки из sessions сохраняются, 
# а данные из hits_agg добавляются по совпадению session_id
df = sessions.merge(hits_agg, on="session_id", how="left")

print("Пропуски до заполнения:")
# Проверка количества пропущенных значений в ключевых столбцах
# Пропуски возникают из-за left join, если в hits_agg нет записи с соответствующим session_id
print(df[["hits_cnt","pages_nunique","actions_nunique","categories_nunique","target"]].isna().sum())

# Заполнение пропусков нулями для ключевых столбцов
for col in ["hits_cnt", "pages_nunique", "actions_nunique", "categories_nunique", "target"]:
    df[col] = df[col].fillna(0)  # Заменяем NaN на 0 для указанных столбцов

# Преобразование столбца target к целочисленному типу
# Это важно, так как target обычно используется как бинарная метка (0/1)
df["target"] = df["target"].astype(int)

print("Пропуски после заполнения:")
# Проверка, что все пропуски устранены
print(df[["hits_cnt","pages_nunique","actions_nunique","categories_nunique","target"]].isna().sum())

print("\nРазмер итогового датафрейма:", df.shape)  # Вывод финальной размерности данных
print("Доля target=1:", df["target"].mean())  # Расчет баланса классов - важный показатель для задач классификации

Комментарий:Этот код визуализирует распределение целевой переменной, что является важным этапом EDA (Exploratory Data Analysis).

In [ ]:
import matplotlib.pyplot as plt

# Создание столбчатой диаграммы распределения значений target
# value_counts(normalize=True) возвращает относительные частоты (доли) вместо абсолютных количеств
df["target"].value_counts(normalize=True).plot(kind="bar")
plt.title("Распределение целевого признака")  # Заголовок графика
plt.ylabel("Доля сессий")  # Подпись оси Y - доля/процент сессий
plt.xlabel("Target")  # Подпись оси X - значения целевой переменной
plt.show()  # Отображение графика

**Комментарий:**  
Большинство сессий содержит целевое действие. Это связано с широким определением таргета, включающим события типа `sub_*` и `start_chat`.


In [ ]:
# Построение boxplot (ящика с усами) для сравнения распределения hits_cnt по группам target
df.boxplot(column="hits_cnt", by="target")
plt.title("Количество событий в сессии по таргету")  # Основной заголовок графика
plt.suptitle("")  # Удаление автоматического заголовка, который добавляет pandas
plt.xlabel("Target")  # Подпись оси X - значения целевой переменной (0 или 1)
plt.ylabel("hits_cnt")  # Подпись оси Y - количество событий в сессии
plt.show()  # Отображение графика

**Комментарий:**  
Сессии с целевым действием характеризуются большим количеством событий. Это подтверждает гипотезу о том, что вовлечённость пользователя напрямую связана с вероятностью совершения целевого действия.

In [ ]:
# Группировка данных по категории устройства и расчет среднего значения target для каждой группы
df.groupby("device_category")["target"].mean().plot(kind="bar")
plt.title("Доля целевых сессий по типу устройства")  # Заголовок: показывает, что анализируем конверсию по устройствам
plt.ylabel("Среднее значение target")  # Ось Y: фактически это доля сессий с target=1 (конверсионная доля)
plt.xlabel("Тип устройства")  # Ось X: категории устройств (mobile, desktop, tablet и т.д.)
plt.show()  # Отображение графика

**Комментарий:**  
Наблюдаются различия в доле целевых сессий между типами устройств, что указывает на влияние устройства пользователя на поведение и конверсию.

## Подход к машинному обучению

В качестве baseline была использована логистическая регрессия как простая и интерпретируемая модель.  
Далее была обучена модель машинного обучения на деревьях решений (RandomForest),
позволяющая учитывать нелинейные зависимости и взаимодействия признаков.

Качество моделей оценивалось по метрике ROC-AUC на отложенной выборке.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Определение признаков:
# Числовые признаки (метрики и счетчики)
num_features = [
    "visit_number",       # Номер визита пользователя
    "hits_cnt",           # Количество событий в сессии
    "pages_nunique",      # Количество уникальных страниц
    "actions_nunique",    # Количество уникальных действий
    "categories_nunique"  # Количество уникальных категорий
]

# Категориальные признаки (дискретные переменные)
cat_features = ["device_category"]  # Тип устройства

# Подготовка матрицы признаков и целевой переменной
X = df[num_features + cat_features]  # Объединение всех признаков
y = df["target"]  # Целевая переменная

# Разделение данных на обучающую и тестовую выборки
# test_size=0.2: 20% данных для тестирования, 80% для обучения
# random_state=42: фиксация случайного состояния для воспроизводимости
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Создание трансформера для предобработки признаков
preprocess = ColumnTransformer(
    transformers=[
        # OneHotEncoder для категориальных признаков
        # handle_unknown="ignore": игнорировать новые категории в тестовых данных
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
        # "passthrough": числовые признаки используются без изменений
        ("num", "passthrough", num_features),
    ]
)

# Создание pipeline: цепочка преобразований и модель
logreg = Pipeline(
    steps=[
        ("prep", preprocess),  # Шаг предобработки
        # Логистическая регрессия с увеличенным числом итераций
        ("clf", LogisticRegression(max_iter=200))
    ]
)

# Обучение модели на тренировочных данных
logreg.fit(X_train, y_train)

# Получение вероятностей для положительного класса на тестовых данных
# predict_proba возвращает вероятности для обоих классов, берем только для класса 1
proba_lr = logreg.predict_proba(X_test)[:, 1]

# Расчет ROC-AUC метрики
# ROC-AUC оценивает способность модели различать классы
auc_lr = roc_auc_score(y_test, proba_lr)

print("Logistic Regression ROC-AUC:", auc_lr)  # Вывод результата

Логистическая регрессия используется как baseline-модель. 
Она демонстрирует хорошее качество, однако ограничена линейной
формой зависимости между признаками и целевой переменной.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Создание подвыборки для обучения (300K строк вместо всего датасета)
# Используется из-за ограничений оперативной памяти (8 ГБ ОЗУ)
# random_state=42 обеспечивает воспроизводимость случайной выборки
df_sample = df.sample(300_000, random_state=42)

# Подготовка признаков и целевой переменной для подвыборки
# ВАЖНО: используем только числовые признаки, категориальные исключены
X = df_sample[num_features]
y = df_sample["target"]

# Разделение на обучающую и тестовую выборки с учетом баланса классов
# stratify=y гарантирует сохранение пропорции классов в разделенных выборках
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Создание модели RandomForest с заданными гиперпараметрами
rf = RandomForestClassifier(
    n_estimators=200,      # Количество деревьев в ансамбле
    max_depth=12,          # Максимальная глубина каждого дерева (ограничение для борьбы с переобучением)
    n_jobs=-1,             # Использование всех ядер процессора для параллельных вычислений
    random_state=42        # Фиксация случайности для воспроизводимости
)

# Обучение модели на тренировочных данных
rf.fit(X_train, y_train)

# Получение вероятностей принадлежности к классу 1 для тестовой выборки
proba_rf = rf.predict_proba(X_test)[:, 1]

# Расчет ROC-AUC метрики для оценки качества модели
auc_rf = roc_auc_score(y_test, proba_rf)

print("RandomForest ROC-AUC:", auc_rf)  # Вывод результата

In [ ]:
import pandas as pd

# Создание DataFrame для сравнения результатов моделей
results = pd.DataFrame({
    "Модель": ["Logistic Regression", "RandomForest"],  # Названия тестируемых моделей
    "ROC-AUC": [auc_lr, auc_rf]  # Значения ROC-AUC метрики для каждой модели
})

results  # Отображение таблицы с результатами (в Jupyter Notebook)

## Выводы

- Данные по событиям (`ga_hits`) были агрегированы до уровня сессии и объединены с таблицей визитов (`ga_sessions`).
- Проведён EDA: исследовано распределение таргета и связь целевого действия с поведенческими признаками.
- В качестве baseline обучена Logistic Regression; далее обучена ML-модель RandomForest на деревьях решений.
- RandomForest показал более высокое качество по ROC-AUC по сравнению с baseline, что объясняется способностью деревьев учитывать нелинейные зависимости и пороговые эффекты.
- Наибольший вклад в прогноз вносят признаки активности пользователя в сессии (количество событий, разнообразие действий и страниц).